In [1]:
!pip install git+https://github.com/alberanid/imdbpy

  Cloning https://github.com/alberanid/imdbpy to c:\users\eric\appdata\local\temp\pip-req-build-cc91rftu


In [2]:
!pip install tqdm

In [3]:
import pandas as pd
import numpy as np
import imdb
import requests
from bs4 import BeautifulSoup

In [4]:
ia = imdb.IMDb()

In [5]:
df_movies = pd.read_csv('scraped_imdb_ph.csv', index_col=0)
df_movies

,Title,Year
0,Insiang,(1976)
1,Ipaglaban mo,(2014– )
2,Just the Way You Are,(2015)
3,Mula sa kung ano ang noon,(2014)
4,Dead Kids,(2019)
...,...,...
19495,Metamorphosis,(VI) (2019)
19496,Quezon's Game,(2018)
19497,My Ex and Whys,(2017)
19498,The Twilight People,(1972)


In [6]:
import re

df_movies['Year'] = df_movies['Year'].map(lambda s: re.search('\d{4}', str(s)).group() if re.search('\d{4}', str(s)) else None)
df_movies.dropna(inplace=True)
df_movies['Year'] = df_movies['Year'].astype(int)

In [ ]:
import imdb
from tqdm import tqdm

ia = imdb.IMDb()

bar = tqdm(df_movies.iloc[0:].iterrows())

for index, row in bar:
    movies = ia.search_movie(row['Title'])
    if movies:
        correct_movie = None
        for movie in movies[:5]:
            bar.set_description(row['Title'] +' '+ str(index))
            m = ia.get_movie(movie.getID())
            if m.get('year') == row['Year']:
                correct_movie = m
            if correct_movie:
                break
        if correct_movie:
            url = ia.get_imdbURL(m)
            src = requests.get(url, headers = {'User-Agent':'Mozilla/5.0 (X11; Linux)'})
            soup = BeautifulSoup(src.content, 'lxml')
            try:
                genre = soup.find('div', {'class': 'subtext'}).text.split('|')[-2].replace('\n', '')
                df_movies.loc[index, 'Genre'] = genre
            except:
                pass
            try:
                plot = soup.find('div', {'class': 'inline canwrap'}).text.replace('\n', '').strip()
                df_movies.loc[index, 'Plot'] = plot
            except:
                pass
            kind = m.get('kind')    
            if kind:
                df_movies.loc[index, 'Kind'] = kind
    if index % 50 == 1:
        df_movies.to_csv('ph_media_updated.csv', index=False)

Forevermore 27: : 27it [03:55, 11.28s/it]                 